# Homework 3 - Kaggle Submission

## Load packages and the data into `bike_train` and `bike_test`.

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, RobustScaler, PolynomialFeatures
from sklearn.model_selection import GridSearchCV, train_test_split
import xgboost as xgb
from datetime import datetime
import seaborn as sns

def load_data(filename):
    """
    Loads the data from the csv file.

    :param filename: name of the csv file
    :return: dataframe
    """
    return pd.read_csv(filename)

bike_train = load_data('Bike_train.csv')
bike_test = load_data('Bike_test.csv')


## Do some data exploration and data cleaning on `bike_training`.

`bike_test` will have these same modifications but we will do it once the model is built. (This is a bit messy, but it's the fastest way to go for now.)

In [5]:
# del humidity = 0
bike_train = bike_train.loc[bike_train['humidity'] != 0, :]

In [6]:
# add day of week column
def get_weekday(row):
    """
    Adds a new column of the day of week from the datetime column.

    :param df: dataframe
    :return: dataframe
    """
    date_str = "{}-{}-{}".format(row["year"].astype(int), row["month"].astype(int), row["day"].astype(int))
    date = datetime.strptime(date_str, '%Y-%m-%d')
    return date.isoweekday()


bike_train["weekday"] = bike_train.apply(get_weekday, axis=1)


Let's see how the data looks like as of now

In [7]:
print(bike_train.columns)

bike_train.head(4)

Index(['daylabel', 'year', 'month', 'day', 'hour', 'season', 'holiday',
       'workingday', 'weather', 'temp', 'atemp', 'humidity', 'windspeed',
       'count', 'weekday'],
      dtype='object')


,daylabel,year,month,day,hour,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,count,weekday
0,1,2011,1,1,0,1,0,0,1.0,9.84,14.395,81.0,0.0,16,6
1,1,2011,1,1,1,1,0,0,1.0,9.02,13.635,80.0,0.0,40,6
2,1,2011,1,1,2,1,0,0,1.0,9.02,13.635,80.0,0.0,32,6
3,1,2011,1,1,3,1,0,0,1.0,9.84,14.395,75.0,0.0,13,6


Bike train weather data has values with decimals ending in ".5", which doesn't make sense


In [8]:
# Bike train weather data has values with decimals ending in ".5", which doesn't make sense
bike_train[bike_train['weather'].astype(str).str.contains('.5')]

,daylabel,year,month,day,hour,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,count,weekday
29,2,2011,1,2,5,1,0,0,2.5,18.04,21.9675,85.5,16.49875,0,7
123,6,2011,1,6,3,1,0,1,1.5,6.56,10.6075,64.0,3.00160,0,4
364,32,2011,2,1,4,1,0,1,2.5,6.15,10.9850,81.0,0.00000,0,2
579,41,2011,2,10,3,1,0,1,2.5,5.74,7.1975,69.5,11.99970,0,4
964,70,2011,3,11,4,1,0,1,1.5,12.71,15.5300,87.0,10.50225,0,5
8726,47,2011,2,16,2,1,0,1,1.5,8.20,10.6075,45.5,9.50060,0,3
8822,75,2011,3,16,2,1,0,1,2.5,11.89,13.6350,100.0,17.50040,0,3
9567,292,2011,10,19,3,4,0,1,2.5,20.91,24.6200,85.5,12.99800,0,3


These are not much and test data doesn't have any of these weird values. I'm just going to drop them

In [9]:
bike_train = bike_train[~bike_train['weather'].astype(str).str.contains('.5')]

Split the data into X and Y in two different dataframes

In [10]:
y_train = bike_train['count']
X_train = bike_train.drop(['count'], axis=1)

Here we are doing a bunch of different things

1. Drop day column. After adding the day of the week {1,2,3,4,5,6,7} for {Monday, Tuesday, Wednesday, Thursday, Friday, Saturday, Sunday}, the `day` column is not useful anymore.
   
2. Make dummies for `year`, `month`, "weekday", `season`, `holiday`, `workingday`, `weather`. Here we are not dropping the first class for each dummy. (We know for linear regression that if we have for example a dummy for gender = male or female, we will only get 1 dummy variable where 1=female and 0=male, leaving `k-1` classes for each dummy . **In this case, for our dummies, we are actually leaving `k` classes**)
   
3. Scale the variables that are not dummy with `RobustScaler`, this is similar to normalizing the data, but performs better when there are outliers

    More info here: https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.RobustScaler.html
    ```
    Scale features using statistics that are robust to outliers.

    This Scaler removes the median and scales the data according to the quantile range (defaults to IQR: Interquartile Range). The IQR is the range between the 1st quartile (25th quantile) and the 3rd quartile (75th quantile).

    Centering and scaling happen independently on each feature by computing the relevant statistics on the samples in the training set. Median and interquartile range are then stored to be used on later data using the transform method.

    Standardization of a dataset is a common requirement for many machine learning estimators. Typically this is done by removing the mean and scaling to unit variance. However, outliers can often influence the sample mean / variance in a negative way. In such cases, the median and the interquartile range often give better results.```

4. Create interactions for all the variables (2 degree polynomial features). **We went from 39 columns of features to 780 columns**

In [11]:
## drop day column
X_train = X_train.drop(['day'], axis=1)

## dummies
X_train_dummies = pd.get_dummies(X_train, columns=["year", "month", "weekday", "season", "holiday", "workingday", "weather"])

## scaling
scaler = RobustScaler()
scaler.fit(X_train_dummies.loc[:, ["temp", "atemp", "humidity", "windspeed"]])

X_train_dummies.loc[:, ["temp", "atemp", "humidity", "windspeed"]] = scaler.transform(
    X_train_dummies.loc[:, ["temp", "atemp", "humidity", "windspeed"]]
)

## interactions
poly = PolynomialFeatures(degree=2, interaction_only=True, include_bias=False)
X_train_dummies_poly = poly.fit_transform(X_train_dummies)

I actually split the train data again to get a validation set. It wasn't really necessary to do this, because we are going to use cross validation later. The val set = 20% of the train set

**Important step** = transform the target variable (y) to $\log\left( y_\text{train} + 1 \right)$


Bad bad practice here: reusing variable names. YOLO

In [83]:
# split data into train and val set
X_train, X_val, y_train, y_val = train_test_split(X_train_dummies_poly, y_train, test_size=0.2, random_state=1706)

y_train_log = np.log(y_train + 1)
y_val_log = np.log(y_val + 1)

# XGBOOST

https://xgboost.readthedocs.io/en/stable/parameter.html

#### Do GridSearchCV / hyperparameter optimization

In [17]:
params = {
    'max_depth': [8],
    'learning_rate': [0.05, 0.07],
    'colsample_bytree': [0.3],
    'n_estimators': [300, 500],
    'gamma': [0.1],
}

xgbr = xgb.XGBRegressor(seed = 1246)
clf = GridSearchCV(estimator=xgbr, param_grid=params, scoring='neg_mean_squared_error', cv=5, verbose=5, n_jobs=-1)


clf.fit(X_train, y_train_log)

Fitting 5 folds for each of 4 candidates, totalling 20 fits


/Users/seba/miniforge3/envs/booth-mlearning/lib/python3.9/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
/Users/seba/miniforge3/envs/booth-mlearning/lib/python3.9/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
/Users/seba/miniforge3/envs/booth-mlearning/lib/python3.9/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
/Users/seba/miniforge3/envs/booth-mlearning/lib/python3.9/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index 

[CV 2/5] END colsample_bytree=0.3, gamma=0.1, learning_rate=0.05, max_depth=8, n_estimators=300;, score=-0.117 total time= 2.5min


/Users/seba/miniforge3/envs/booth-mlearning/lib/python3.9/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


[CV 3/5] END colsample_bytree=0.3, gamma=0.1, learning_rate=0.05, max_depth=8, n_estimators=300;, score=-0.115 total time= 2.5min
[CV 1/5] END colsample_bytree=0.3, gamma=0.1, learning_rate=0.05, max_depth=8, n_estimators=300;, score=-0.120 total time= 2.5min


...

[CV 4/5] END colsample_bytree=0.3, gamma=0.1, learning_rate=0.05, max_depth=8, n_estimators=500;, score=-0.108 total time= 3.3min


/Users/seba/miniforge3/envs/booth-mlearning/lib/python3.9/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


[CV 1/5] END colsample_bytree=0.3, gamma=0.1, learning_rate=0.07, max_depth=8, n_estimators=300;, score=-0.115 total time= 1.2min
[CV 3/5] END colsample_bytree=0.3, gamma=0.1, learning_rate=0.07, max_depth=8, n_estimators=300;, score=-0.110 total time= 1.2min
[CV 2/5] END colsample_bytree=0.3, gamma=0.1, learning_rate=0.07, max_depth=8, n_estimators=300;, score=-0.114 total time= 1.2min
[CV 4/5] END colsample_bytree=0.3, gamma=0.1, learning_rate=0.07, max_depth=8, n_estimators=300;, score=-0.116 total time= 1.2min
[CV 5/5] END colsample_bytree=0.3, gamma=0.1, learning_rate=0.07, max_depth=8, n_estimators=300;, score=-0.112 total time= 1.2min
[CV 1/5] END colsample_bytree=0.3, gamma=0.1, learning_rate=0.07, max_depth=8, n_estimators=500;, score=-0.109 total time= 1.1min
[CV 3/5] END colsample_bytree=0.3, gamma=0.1, learning_rate=0.07, max_depth=8, n_estimators=500;, score=-0.105 total time= 1.1min
[CV 2/5] END colsample_bytree=0.3, gamma=0.1, learning_rate=0.07, max_depth=8, n_estimator

/Users/seba/miniforge3/envs/booth-mlearning/lib/python3.9/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


GridSearchCV(cv=5,
             estimator=XGBRegressor(base_score=None, booster=None,
                                    colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None,
                                    enable_categorical=False, gamma=None,
                                    gpu_id=None, importance_type=None,
                                    interaction_constraints=None,
                                    learning_rate=None, max_delta_step=None,
                                    max_depth=None, min_child_weight=None,
                                    missing=nan, monotone_constraints=None,
                                    n...
                                    num_parallel_tree=None, predictor=None,
                                    random_state=None, reg_alpha=None,
                                    reg_lambda=None, scale_pos_weight=None,
                                 

In [92]:
## saving to not lose on restart of kernel
## obtained doing clf.best_params_
best_params = {'colsample_bytree': 0.3,
 'gamma': 0.1,
 'learning_rate': 0.05,
 'max_depth': 8,
 'n_estimators': 500}

In [85]:
xgb2 = xgb.XGBRegressor(seed=1246, **best_params)
xgb2.fit(X_train, y_train_log)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.3, enable_categorical=False,
             gamma=0.1, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.05, max_delta_step=0,
             max_depth=8, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=500, n_jobs=10,
             num_parallel_tree=1, predictor='auto', random_state=1246,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=1246,
             subsample=1, tree_method='exact', validate_parameters=1,
             verbosity=None)

In [103]:
pd.DataFrame(xgb2.feature_importances_, index=poly.get_feature_names_out(X_train_dummies.columns)).sort_values(by=0, ascending=False)

,0
season_3,0.150295
hour weather_1.0,0.036266
hour,0.035352
hour year_2012,0.032660
hour holiday_0,0.031941
...,...
month_3 month_9,0.000000
month_3 month_10,0.000000
month_3 month_11,0.000000
month_10 holiday_1,0.000000


In [87]:
np.expm1(xgb2.predict(X_val)).round(0)

## RMSE
np.sqrt(np.mean(np.square(np.expm1(xgb2.predict(X_val)).round(0) - y_val)))

38.09803866944748

In [104]:
np.mean(np.square(xgb2.predict(X_val) - y_val_log))

0.07938316873909343

#### Transform test dataset 

Here we are applying all the transformations we did on the train data. We are using the same models we built earlier, but we are using the test data.

In [89]:


bike_test["weekday"] = bike_test.apply(get_weekday, axis=1)

## drop day column
X_test = bike_test.drop(['day'], axis=1)

## dummies
X_test_dummies = pd.get_dummies(X_test, columns=["year", "month", "weekday", "season", "holiday", "workingday", "weather"])


X_test_dummies.loc[:, ["temp", "atemp", "humidity", "windspeed"]] = scaler.transform(
    X_test_dummies.loc[:, ["temp", "atemp", "humidity", "windspeed"]]
)

X_test_dummies.rename(columns={'weather_1': 'weather_1.0', 'weather_2': "weather_2.0", 'weather_3': "weather_3.0", 'weather_4': "weather_4.0"}, inplace=True)

## interactions
X_test_dummies_poly = poly.transform(X_test_dummies)


In [90]:
y_pred = xgb2.predict(X_test_dummies_poly)
y_pred_expm1 = np.expm1(y_pred).round(0).astype(int)

pd.DataFrame([list(range(1, y_pred_expm1.shape[0]+1)), y_pred_expm1], index=['Id', 'count']).T.to_csv('submission9.csv', index=False)